In [3]:
!pip install langchain langchain-google-genai langchain-text-splitters langchain-community langchain-core langchain-huggingface sentence-transformers bs4 pypdf requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.1.0
    Uninstalling langchain-core-1.1.0:
      Successfully uninstalled langchain-core-1.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are install

In [2]:
import os
import bs4
import requests
from langchain.tools import tool
from langchain.agents import AgentState
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.messages import MessageLikeRepresentation
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [6]:
pdf_url = "https://peraturan.bpk.go.id/Download/46205/PP%20No.%2040%20Th%201996.pdf"
pdf_path = "PP_No_40_Th_1996.pdf"

if not os.path.exists(pdf_path):
    print(f"Downloading PDF from {pdf_url}...")
    response = requests.get(pdf_url)
    with open(pdf_path, 'wb') as f:
        f.write(response.content)
    print(f"PDF downloaded successfully to {pdf_path}")
else:
    print(f"PDF already exists at {pdf_path}")

PDF downloaded successfully to PP_No_40_Th_1996.pdf


In [7]:
loader = PyPDFLoader(pdf_path)
docs = loader.load()
print(f"Loaded {len(docs)} pages from PDF")

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vector_store = InMemoryVectorStore(embeddings)
_ = vector_store.add_documents(documents=all_splits)

Loaded 64 pages from PDF


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [9]:
def prompt_with_context(state: AgentState) -> list[MessageLikeRepresentation]:
    last_query = state["messages"][-1].text
    retrieved_docs = vector_store.similarity_search(last_query)

    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

    system_message = (
        "Anda adalah asisten hukum dan ahli geospasial yang membantu menjawab pertanyaan berdasarkan"
        f"\n\n{docs_content}"
    )

    return [{"role": "system", "content": system_message}, *list(state["messages"])]

tools = [retrieve_context]

In [11]:
llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai", api_key=userdata.get('GOOGLE_API_KEY'))

In [12]:
checkpointer = MemorySaver()

In [13]:
agent = create_react_agent(
    model=llm,
    tools=tools,
    prompt=prompt_with_context,
    checkpointer=checkpointer
)

/tmp/ipython-input-1421255084.py:1: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


In [14]:
query = "Tanah yang dapat diberikan dengan Hak Guna Usaha adalah tanah Negara tersebut dalam pasal berapa?"

In [15]:
config = {"configurable": {"thread_id": "1"}}

for step in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    config=config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Tanah yang dapat diberikan dengan Hak Guna Usaha adalah tanah Negara tersebut dalam pasal berapa?
================================== Ai Message ==================================

[{'type': 'text', 'text': 'Tanah yang dapat diberikan dengan Hak Guna Usaha adalah tanah Negara tersebut dalam Pasal 4 ayat (1).', 'extras': {'signature': 'CtoDAXLI2nxjjySA0gCl4r2QnmJOeOXdxOGXoUq+geMhzXNhoUsLI9/ECb0IGD+eg7rHsWo+OQCzyEPzXECSaaYLJs60p+ffLVUtRsmlVtXiDs1thQXk6jiqjEM7nr6r6e7PLZADF6f7/AqfrSC1MqDQ8tiYax1j7kO++X8Ox6o8CD5Bm01RJJpadjxmWRTF/aWSBIEyUaZq0qSub1EPbPHIoB72e4+QH5L8lmnYyS3iDS6ozqxWqJxKlfkIhyP6Wox7Q5cP9Uh3kPMaigXXgSpJ5kwd8hoRhf3GYBS9d2CDsp6fgyVGMlw46qOjzBjvCYGvaadJ15a3uRhbFLQjWaok+C4ubNzrEbeJHU7bPNw+AZpjWCdASU+8+WubQZKRro++9D3hlIb5f/glJN8gsjkO1wMisF+LSUtVEfRVIzhR5+FJPG2x4ognGo+x0TxY3poF1Gbty20kfITEsMp4PYIDK3zm7DIHi0EHad/xil+flbbXGg+SYAj6yPFWZroSFepv8Vdty29EoGiAUkGkA+25I7WIGc3KxCfjVqIn0wfFbZAbf6UzDYvViu8ctuucitY/G7